In [1]:
import pandas as pd
import numpy as np
import spacy
import scipy as sp
import sklearn



In [36]:
# !pip install spacy

In [37]:
df = pd.read_csv('manually_fixed.csv')


In [38]:
df.head()

newdf= df[['failure_mode_tm', 'long_text_notification']]

In [39]:
newdf.head()
newdf.shape

(1532, 2)

In [40]:
df2 = newdf.dropna()
df.shape
#newdf1.head()

(1532, 31)

In [41]:
df2.shape

(845, 2)

In [42]:
df2.head()

,failure_mode_tm,long_text_notification
1,Worn,18.04.2016 01:58:07 UTC Ronald Piggott (PIGGR9...
2,Worn,06.06.2016 14:34:43 UTC Ronald Piggott (PIGGR9...
3,Worn,27.07.2016 13:34:51 UTC Owen Nugent (NUGEO91) ...
4,Failed,15.10.2013 02:57:03 Stuart Mau (MAUST) Defi...
5,Worn,09.01.2017 15:54:12 UTC Kris Day (DAYKR9) Phon...


In [43]:
myvar = df2['long_text_notification'].loc[3]
myvar

'27.07.2016 13:34:51 UTC Owen Nugent (NUGEO91)    Brief description of what occurred (e.g. basic sequence of events and impacts avoiding the use of personnel names):  Replace Gear Selector   *     Define work requirements; including known materials specialist labour special tools that are required:  10855529 x 1 12-4 for 3 hours   *     *    '

In [44]:
# Preprocessing 3:
# Remove web related garbage
# Remove any words with anything other than characters (a-z|A-Z)

def remove_garbage(list_words):
    new_list = []
    for word in list_words:
        if word.isalpha(): new_list.append(word)        
    return new_list


In [45]:
def remove_template(str_input):
    var2 = str_input[str_input.find('Brief'):]
    var3 = var2[var2.find(':') + 1:]
    var4 = var3[:var3.find("Define")] + var3[var3.find(':')+ 1:]
    var5 = remove_garbage(var4.split())
    var6 = [ x.lower() for x in  var5 if len(x) >1 ]
    var7 = ' '.join(var6)
    
    return var7
    

In [46]:
df2['clean_text'] = df2.long_text_notification.apply(lambda words: remove_template(words))

/home/profnick/anaconda3/envs/ztdl/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [47]:
df2.head()

,failure_mode_tm,long_text_notification,clean_text
1,Worn,18.04.2016 01:58:07 UTC Ronald Piggott (PIGGR9...,gear changer not working needs replacing elect...
2,Worn,06.06.2016 14:34:43 UTC Ronald Piggott (PIGGR9...,gear selector playing up needs replacing elect...
3,Worn,27.07.2016 13:34:51 UTC Owen Nugent (NUGEO91) ...,replace gear selector for hours
4,Failed,15.10.2013 02:57:03 Stuart Mau (MAUST) Defi...,
5,Worn,09.01.2017 15:54:12 UTC Kris Day (DAYKR9) Phon...,whilst completing sos operator reported fault ...


In [48]:
df3 = df2[df2['clean_text'].apply(lambda x: len(x)>0)]

#df3.to_csv("sample.csv")
df3.head()

,failure_mode_tm,long_text_notification,clean_text
1,Worn,18.04.2016 01:58:07 UTC Ronald Piggott (PIGGR9...,gear changer not working needs replacing elect...
2,Worn,06.06.2016 14:34:43 UTC Ronald Piggott (PIGGR9...,gear selector playing up needs replacing elect...
3,Worn,27.07.2016 13:34:51 UTC Owen Nugent (NUGEO91) ...,replace gear selector for hours
5,Worn,09.01.2017 15:54:12 UTC Kris Day (DAYKR9) Phon...,whilst completing sos operator reported fault ...
7,Worn,09.01.2018 13:23:33 UTC Rory Bean (BEANRX) ...,faulty selector lever changed out selector lev...


In [49]:
#Remove Stop words
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer, WordNetLemmatizer
porter_stemmer = PorterStemmer()
stemmer = SnowballStemmer("english")
stop = stopwords.words('english')

df3['stop_words_removed'] = df3['clean_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))


/home/profnick/anaconda3/envs/ztdl/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [50]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/profnick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [51]:
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df3['stemmed'] = df3['stop_words_removed'].apply(stem_sentences)

/home/profnick/anaconda3/envs/ztdl/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [52]:
df3.head()

,failure_mode_tm,long_text_notification,clean_text,stop_words_removed,stemmed
1,Worn,18.04.2016 01:58:07 UTC Ronald Piggott (PIGGR9...,gear changer not working needs replacing elect...,gear changer working needs replacing elect hrs,gear changer work need replac elect hrs
2,Worn,06.06.2016 14:34:43 UTC Ronald Piggott (PIGGR9...,gear selector playing up needs replacing elect...,gear selector playing needs replacing elect hrs,gear selector play need replac elect hrs
3,Worn,27.07.2016 13:34:51 UTC Owen Nugent (NUGEO91) ...,replace gear selector for hours,replace gear selector hours,replac gear selector hour
5,Worn,09.01.2017 15:54:12 UTC Kris Day (DAYKR9) Phon...,whilst completing sos operator reported fault ...,whilst completing sos operator reported fault ...,whilst complet sos oper report fault gear leav...
7,Worn,09.01.2018 13:23:33 UTC Rory Bean (BEANRX) ...,faulty selector lever changed out selector lev...,faulty selector lever changed selector lever f...,faulti selector lever chang selector lever fit...


In [53]:
#Building the model
df3.to_csv('hello.csv')

In [2]:
df3 = pd.read_csv('hello.csv')

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer


tp_vectorizer = TfidfVectorizer(min_df = 1, binary = True)

train_X = df3.iloc[:, :]['stemmed']
tp_train_X = tp_vectorizer.fit_transform(train_X)
train_Y = df3.iloc[:, :]['failure_mode_tm']


In [4]:
df3['failure_mode_tm'].value_counts()

worn          185
Worn          175
Failed        114
Inoperable     17
damaged        13
Damaged        10
Fails           5
failed          5
output          3
Wears           1
burnt           1
cracked         1
Name: failure_mode_tm, dtype: int64

In [5]:
df3['target'] = df3['failure_mode_tm'].apply(lambda x: x.lower())

df3['target_2'] = df3['target'].str.replace('fails', 'failed')

In [6]:
df3.head()
df3['target_2'].value_counts()

worn          360
failed        124
damaged        23
inoperable     17
output          3
burnt           1
cracked         1
wears           1
Name: target_2, dtype: int64

In [7]:
df4 = df3.reset_index(drop = True)

In [8]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 8 columns):
Unnamed: 0                530 non-null int64
failure_mode_tm           530 non-null object
long_text_notification    530 non-null object
clean_text                530 non-null object
stop_words_removed        530 non-null object
stemmed                   530 non-null object
target                    530 non-null object
target_2                  530 non-null object
dtypes: int64(1), object(7)
memory usage: 33.2+ KB


In [13]:
from sklearn import svm
#clf = svm.SVC(gamma='scale')

clf= svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
#clf.fit(tp_train_X, train_Y)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer


tp_vectorizer = TfidfVectorizer(min_df = 1)

train_X = df4.iloc[:, :]['stop_words_removed']
tp_train_X = tp_vectorizer.fit_transform(train_X)
train_Y = df4.iloc[:, :]['target_2']

In [21]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
 
mykFold = KFold(n_splits=5, random_state=None, shuffle=False)
for train_index, test_index in mykFold.split(tp_train_X):
#    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = tp_train_X[train_index], tp_train_X[test_index]
    y_train, y_test = train_Y[train_index], train_Y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc= accuracy_score(y_test, y_pred)
    print(acc)
    break

0.7075471698113207


In [250]:
newdf  = pd.read_csv('sample_test.csv')
newdf.head()

,Unnamed: 0,failure_mode_tm,stop_words_removed
0,1,Worn,gear changer working needs replacing elect hrs
1,2,Worn,gear selector playing needs replacing elect hrs
2,3,Worn,replace gear selector hours
3,5,Worn,whilst completing sos operator reported fault ...
4,7,Worn,faulty selector lever changed selector lever f...


In [259]:
#On new data

tp_test_X = tp_vectorizer.transform(test_X)
clf.predict(tp_test_X)

array(['worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn', 'worn',
       'worn', 'worn', 'worn', 'worn', 'worn'], dtype=object)

In [264]:
test_X = newdf['stop_words_removed']
tp_test_X = tp_vectorizer.transform(test_X)
predictions = clf.predict(tp_test_X)
confidence = clf.predict_proba(tp_test_X)

In [ ]:
T#this is how the final output shoukd be
newdf['predictions'] = predictions
#newdf['Probability'] = confidence

newdf

In [ ]:
type(confidence)
#print(confidence.classes_[order[:, -5:]])
df = pd.DataFrame(confidence)
df

## RANDOM FOREST

In [24]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators=600, max_depth=300, max_features='sqrt')

In [25]:
# Import the model we are using
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

mykFold = KFold(n_splits=5, random_state=None, shuffle=False)
for train_index, test_index in mykFold.split(tp_train_X):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = tp_train_X[train_index], tp_train_X[test_index]
    y_train, y_test = train_Y[train_index], train_Y[test_index]
    # Train the model on training data
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    acc= accuracy_score(y_test, y_pred)
    print(acc)
    break

0.7169811320754716


## NAIVE BAYESIAN

In [37]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
gnb1 = GaussianNB()
gnb2 = BernoulliNB()
gnb3 = MultinomialNB()

In [38]:
# Import the model we are using
# Guassian
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

mykFold = KFold(n_splits=5, random_state=None, shuffle=False)
for train_index, test_index in mykFold.split(tp_train_X):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = tp_train_X[train_index], tp_train_X[test_index]
    y_train, y_test = train_Y[train_index], train_Y[test_index]
    # Train the model on training data
    gnb1.fit(X_train.todense(), y_train)
    y_pred = gnb1.predict(X_test.todense())
    acc= accuracy_score(y_test, y_pred)
    print(acc)
    break

0.5377358490566038


In [39]:
# Import the model we are using
# Bernoulli
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

mykFold = KFold(n_splits=5, random_state=None, shuffle=False)
for train_index, test_index in mykFold.split(tp_train_X):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = tp_train_X[train_index], tp_train_X[test_index]
    y_train, y_test = train_Y[train_index], train_Y[test_index]
    # Train the model on training data
    gnb2.fit(X_train.todense(), y_train)
    y_pred = gnb2.predict(X_test.todense())
    acc= accuracy_score(y_test, y_pred)
    print(acc)
    break

0.7452830188679245


In [40]:
# Import the model we are using
# Multinomial

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

mykFold = KFold(n_splits=5, random_state=None, shuffle=False)
for train_index, test_index in mykFold.split(tp_train_X):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = tp_train_X[train_index], tp_train_X[test_index]
    y_train, y_test = train_Y[train_index], train_Y[test_index]
    # Train the model on training data
    gnb3.fit(X_train.todense(), y_train)
    y_pred = gnb3.predict(X_test.todense())
    acc= accuracy_score(y_test, y_pred)
    print(acc)
    break

0.7358490566037735


## DECISION TREES

In [33]:
from sklearn.tree import DecisionTreeClassifier


tree = DecisionTreeClassifier(criterion = "entropy", splitter = "random", max_depth = 2,  min_samples_split = 5,
                              min_samples_leaf = 2, max_features = 2)


In [35]:
mykFold = KFold(n_splits=5, random_state=None, shuffle=False)
for train_index, test_index in mykFold.split(tp_train_X):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = tp_train_X[train_index], tp_train_X[test_index]
    y_train, y_test = train_Y[train_index], train_Y[test_index]
    # Train the model on training data
    tree.fit(X_train, y_train)
    y_pred = tree.predict(X_test)
    acc= accuracy_score(y_test, y_pred)
    print(acc)
    break

0.7075471698113207


#Further Instructions:

Write the final output of the predictions to a CSV file. In the output the predicted class, the propability (condifdence of the prediction) should be appneded to the test data csv file as you have seen above in the dummy example

For this task:

**Try and build about 3 or 4 different models (ignore basic models like SVM, Logistic Regression, Random Forest), I have already build those models. We will use these models as baseline

**IF you want to use these models then you can try an esemble model where you stack 3 or four different classifiers to see what we can get.

**But focus on models that use neural nets, LSTM etc like what we discussed the other time.

**Behind schedule for this competetion so Please get me something ASAP. 

**Create a git hub repo with all the instructions in codes and it ans share it with me.



## BERT SENTENCE CLASSIFICATION

In [62]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from pandas import Dataframe

In [63]:
le = LabelEncoder()
 
df = pd.read_csv("hello.csv")
df['target'] = df['failure_mode_tm'].apply(lambda x: x.lower())

df['target_2'] = df['target'].str.replace('fails', 'failed')


In [64]:
df.head()

,Unnamed: 0,failure_mode_tm,long_text_notification,clean_text,stop_words_removed,stemmed,target,target_2
0,1,Worn,18.04.2016 01:58:07 UTC Ronald Piggott (PIGGR9...,gear changer not working needs replacing elect...,gear changer working needs replacing elect hrs,gear changer work need replac elect hrs,worn,worn
1,2,Worn,06.06.2016 14:34:43 UTC Ronald Piggott (PIGGR9...,gear selector playing up needs replacing elect...,gear selector playing needs replacing elect hrs,gear selector play need replac elect hrs,worn,worn
2,3,Worn,27.07.2016 13:34:51 UTC Owen Nugent (NUGEO91) ...,replace gear selector for hours,replace gear selector hours,replac gear selector hour,worn,worn
3,5,Worn,09.01.2017 15:54:12 UTC Kris Day (DAYKR9) Phon...,whilst completing sos operator reported fault ...,whilst completing sos operator reported fault ...,whilst complet sos oper report fault gear leav...,worn,worn
4,7,Worn,09.01.2018 13:23:33 UTC Rory Bean (BEANRX) ...,faulty selector lever changed out selector lev...,faulty selector lever changed selector lever f...,faulti selector lever chang selector lever fit...,worn,worn


In [65]:
# Creating train and dev dataframes according to BERT
df_bert = pd.DataFrame({'index':df['Unnamed: 0'],
            'label':le.fit_transform(df['target_2']),
            'alpha':['a']*df.shape[0],
            'text':df['stop_words_removed'].replace(r'\n',' ',regex=True)})

In [66]:
# Train and dev set
df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.01)

In [69]:
# Creating test dataframe according to BERT
df_test = pd.read_csv("sample_test_data.csv")
# df_test.head()
df_bert_test = pd.DataFrame({'':df_test['Unnamed: 0'],
                 'text':df_test['stop_words_removed'].replace(r'\n',' ',regex=True)})

In [70]:
# Saving dataframes to .tsv format as required by BERT
df_bert_train.to_csv('data/train.tsv', sep='\t', index=False, header=False)
df_bert_dev.to_csv('data/dev.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('data/test.tsv', sep='\t', index=False, header=True)